In [ ]:
import numpy as np
import matplotlib.pylab as plt
from geopandas import GeoDataFrame
import geopandas as gp
from fiona.crs import from_epsg
import os
%matplotlib inline

In [ ]:
#import zip file
os.system("curl -O http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nycb2010_16d.zip")
os.system("mv nycb2010_16d.zip " + os.getenv("PUIDATA"))

In [ ]:
#unzip file and move to PUIdata
os.system('unzip ' + os.getenv("PUIDATA") + '/' + 'nycb2010_16d.zip')
os.system("mv nycb2010_16d " + os.getenv("PUIDATA"))

In [ ]:
#read shapefile
shape_path=(os.getenv("PUIDATA") + '/' + 'nycb2010_16d' + '/' + 'nycb2010.shp')
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.crs = from_epsg(2263)

In [ ]:
ct_shape.head()

In [ ]:
# Extract Brooklyn shape polygons and convert the coordinates to lat/lon
ct_latlon = ct_shape.to_crs(epsg=4326)
ct_latlon = ct_latlon[ct_latlon.BoroName == 'Brooklyn']

In [ ]:
ct_latlon.head()

In [ ]:
# CUSP coordinates from http://www.gps-coordinates.net/
#Create a Point shapely geometry with the coordinates of CUSP
cusp = shapely.geometry.Point(-73.986405,40.693216)

In [ ]:
#Loop over the Brookly polygons (nearly 10000) and assess find in which geometry the polygon is contained

cusp_index = 0
for i, ct in zip(ct_latlon.index, ct_latlon.geometry):
    if shapely.geometry.asShape(ct).contains(cusp):
        print 'CUSP',ct
        cusp_index = i

In [ ]:
#save the index of the polygon that contains the CUSP
cusp_index

In [ ]:
# Plot one every 10 polygons of brooklyn 
selected_index = np.arange(ct_latlon.index.min(),ct_latlon.index.max(),10)

f, ax = plt.subplots(figsize=(10, 10))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# only plot non-NA rows
ct_latlon.ix[selected_index].dropna().plot(figsize=(15,10),
                                          cmap='Greys', ax=ax)

# plot CUSP shapefile with red color
GeoDataFrame(ct_latlon.ix[cusp_index]).T.plot(ax=ax, color='r')

# plot CUSP point
plt.scatter(-73.986405,40.693216, s=100, c='b', alpha=0.3)
plt.xlim(-74.05,-73.85)
